# Convergence Figure CRANE

### STEP 1. WOS Tranformation

- use citation-iot/ai-robot-clarivate.sql to extract WOS_id and WOS_ref
- add dataset column
- create a list of unique ids per target (wos_id) and source (ref_id) for a specific domain
- combine three datasets
- add 6 columns with binary values s.ai, s.iot, s.rob, t.ai, t.iot, t.rob. Use lists of ids and target ids to assign 1 or 0
- inspect and remove the self-looping source-target
    - Note: source-target-looping target_id does not have 'WOS:' at the beginning of the id. Remove those records.
- add year and citation information to wos_id using ISI files exported from MAC
- add year to ref_id
- add edge_id (combine wos and ref id separated by hyphen)

In [38]:
import pandas as pd
import numpy as np
import csv
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [39]:
# Load CSV Files
ai = pd.read_csv('data/citation-ai-index-clarivate.csv', encoding='latin1', sep=",")
rob = pd.read_csv('data/citation-robot-index-clarivate.csv', encoding='latin1', sep=",")
iot = pd.read_csv('data/citation-iot-index-clarivate.csv', encoding='latin1', sep=",")

In [40]:
ai['dataset'] = 'ai'
rob['dataset'] = 'rob'
iot['dataset'] = 'iot'

In [41]:
ai_wos = set(ai.id)
ai_ref = set(ai.ref_id)
rob_wos = set(rob.id)
rob_ref = set(rob.ref_id)
iot_wos = set(iot.id)
iot_ref = set(iot.ref_id)

In [42]:
df = pd.concat([ai, rob, iot], ignore_index=True)

In [43]:
df.head()

,id,ref_id,dataset
0,WOS:000403744000004,000403744000004.38,ai
1,WOS:000403744000004,000403744000004.9,ai
2,WOS:000403744000004,000403744000004.23,ai
3,WOS:000403744000004,000403744000004.4,ai
4,WOS:000403744000004,000403744000004.6,ai


In [44]:
df['s.ai'] = df['id'].apply(lambda x: 1 if x in ai_wos else 0)
df['s.rob'] = df['id'].apply(lambda x: 1 if x in rob_wos else 0)
df['s.iot'] = df['id'].apply(lambda x: 1 if x in iot_wos else 0)

In [45]:
df['t.ai'] = df['ref_id'].apply(lambda x: 1 if x in ai_wos else 0)
df['t.rob'] = df['ref_id'].apply(lambda x: 1 if x in rob_wos else 0)
df['t.iot'] = df['ref_id'].apply(lambda x: 1 if x in iot_wos else 0)

In [46]:
df = df.drop(df[df['ref_id'].str.contains('^[0-9]',regex=True)==True].index)

### Upload ISI files

In [47]:
# Load CSV Files [I changed the column names '# Citations' > Citations, 'Publication Year' > Year, 'WoS ID' > wos_id]
ai_isi = pd.read_csv('data/ISI-Publications-ai.csv', encoding='latin1', sep=",")
rob_isi = pd.read_csv('data/ISI-Publications-Robotics.csv', encoding='latin1', sep=",")
iot_isi = pd.read_csv('data/ISI-Publications-IOT.csv', encoding='latin1', sep=",")

In [48]:
df2 = pd.concat([ai_isi, rob_isi, iot_isi], ignore_index=True)

In [49]:
df3 = df.merge(df2, how='left', on=['id'])

In [50]:
df3 = df3.drop(['Discipline','Subdiscipline', 'Title','Authors','Journal'], axis=1)

In [51]:
df3['edge_id'] = df3[['id', 'ref_id']].apply(lambda x: '-'.join(x.astype(str)), axis=1)

In [52]:
df4 = df3.drop(['ref_id','dataset', 's.ai','s.rob','s.iot','t.ai', 't.rob','t.iot','Citations','edge_id'], axis=1)

In [53]:
df4.tail()

,id,Year
506096,WOS:000401664100004,2016
506097,WOS:000401664100004,2016
506098,WOS:000401664100004,2016
506099,WOS:000401664100004,2016
506100,WOS:000401664100004,2016


In [54]:
df4.columns = ['ref_id', 'TargetYear']

In [55]:
df5 = pd.merge(df4,df3,  on=['ref_id'])

In [56]:
df5.head()

,ref_id,TargetYear,id,dataset,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot,Year,Citations,edge_id
0,WOS:000289029800005,2011,WOS:000385595200022,ai,1,0,0,1,0,0,2016,15,WOS:000385595200022-WOS:000289029800005
1,WOS:000289029800005,2011,WOS:000385595200022,ai,1,0,0,1,0,0,2016,15,WOS:000385595200022-WOS:000289029800005
2,WOS:000289029800005,2011,WOS:000385595200022,ai,1,0,0,1,0,0,2016,15,WOS:000385595200022-WOS:000289029800005
3,WOS:000289029800005,2011,WOS:000385595200022,ai,1,0,0,1,0,0,2016,15,WOS:000385595200022-WOS:000289029800005
4,WOS:000289029800005,2011,WOS:000385595200022,ai,1,0,0,1,0,0,2016,15,WOS:000385595200022-WOS:000289029800005


In [57]:
df5.columns.tolist()

['ref_id',
 'TargetYear',
 'id',
 'dataset',
 's.ai',
 's.rob',
 's.iot',
 't.ai',
 't.rob',
 't.iot',
 'Year',
 'Citations',
 'edge_id']

In [58]:
cols = ['edge_id', 'id','ref_id', 's.ai','s.rob','s.iot','t.ai','t.rob','t.iot','dataset','Year','TargetYear','Citations']

In [59]:
df5 = df5[cols]

In [60]:
is_t_iot_year_less_2010 = df5[(df5['TargetYear']< 2010) & (df5['t.iot']==1)]

In [61]:
is_t_iot_year_less_2010.groupby('TargetYear').count()

,edge_id,id,ref_id,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot,dataset,Year,Citations
TargetYear,,,,,,,,,,,,
2006,135,135,135,135,135,135,135,135,135,135,135,135
2007,155,155,155,155,155,155,155,155,155,155,155,155
2008,254,254,254,254,254,254,254,254,254,254,254,254
2009,4078,4078,4078,4078,4078,4078,4078,4078,4078,4078,4078,4078


In [62]:
df5

,edge_id,id,ref_id,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot,dataset,Year,TargetYear,Citations
0,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
1,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
2,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
3,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
4,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085799,WOS:000403205200025-WOS:000383222000014,WOS:000403205200025,WOS:000383222000014,0,0,1,0,0,1,iot,2017,2016,2
1085800,WOS:000418627700013-WOS:000406590900075,WOS:000418627700013,WOS:000406590900075,0,0,1,0,0,1,iot,2017,2016,22
1085801,WOS:000418627700013-WOS:000406590900075,WOS:000418627700013,WOS:000406590900075,0,0,1,0,0,1,iot,2017,2016,22
1085802,WOS:000418627700013-WOS:000406590900075,WOS:000418627700013,WOS:000406590900075,0,0,1,0,0,1,iot,2017,2016,22


In [63]:
df5['s.sum'] = df5['s.ai'] + df5['s.rob'] + df5['s.iot']
df5['t.sum'] = df5['t.ai'] +df5['t.rob']+ df5['t.iot'] 

In [64]:
df6 = df5[(df5['s.sum']==1) & (df5['t.sum']==1)]

In [65]:
df6 = df6.drop(['s.sum','t.sum'], axis=1)

In [66]:
df6.to_csv('convergence_id_ref.csv', index=False)

### Step 2. Conver to Network
- import convergence_id_ref.csv
- parse df3
- create Source and Target nodes and edges

In [67]:
df5 = pd.read_csv('convergence_id_ref.csv', encoding='latin1', sep=",")

/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
df5 = df5.drop_duplicates()

In [69]:
len(df5.id)

29012

In [70]:
df5.to_csv('convergence.csv', index=False)

In [71]:
df5.head()

,edge_id,id,ref_id,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot,dataset,Year,TargetYear,Citations
0,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
36,WOS:000403632000005-WOS:000291123500010,WOS:000403632000005,WOS:000291123500010,1,0,0,1,0,0,ai,2017,2011,2
113,WOS:000416790500071-WOS:000413946600014,WOS:000416790500071,WOS:000413946600014,0,0,1,1,0,0,iot,2017,2017,4
150,WOS:000372644300043-WOS:000290536400012,WOS:000372644300043,WOS:000290536400012,1,0,0,1,0,0,ai,2016,2011,1
161,WOS:000408751100012-WOS:000400822900194,WOS:000408751100012,WOS:000400822900194,1,0,0,1,0,0,ai,2017,2017,3


In [72]:
wosYears = {}
wosCitations = {}
AItoROB = {}
IOTtoAI = {}
ROBtoIOT = {}
ROBtoAI = {}
AItoIOT = {}
IOTtoROB = {}

Nodes = []
NodeDict = {}
Edges = []

In [73]:
df5.head()

,edge_id,id,ref_id,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot,dataset,Year,TargetYear,Citations
0,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
36,WOS:000403632000005-WOS:000291123500010,WOS:000403632000005,WOS:000291123500010,1,0,0,1,0,0,ai,2017,2011,2
113,WOS:000416790500071-WOS:000413946600014,WOS:000416790500071,WOS:000413946600014,0,0,1,1,0,0,iot,2017,2017,4
150,WOS:000372644300043-WOS:000290536400012,WOS:000372644300043,WOS:000290536400012,1,0,0,1,0,0,ai,2016,2011,1
161,WOS:000408751100012-WOS:000400822900194,WOS:000408751100012,WOS:000400822900194,1,0,0,1,0,0,ai,2017,2017,3


In [74]:
for index, row in df5.iterrows():
    wosID = str(row[1])
    year = str(row[10])
    wosYears[wosID] = year

In [75]:
#wosYears = dict(zip(df5.id, df5.Year))

In [76]:
len(wosYears)

10877

In [77]:
#AItoROB

In [80]:
df5

,edge_id,id,ref_id,s.ai,s.rob,s.iot,t.ai,t.rob,t.iot,dataset,Year,TargetYear,Citations
0,WOS:000385595200022-WOS:000289029800005,WOS:000385595200022,WOS:000289029800005,1,0,0,1,0,0,ai,2016,2011,15
36,WOS:000403632000005-WOS:000291123500010,WOS:000403632000005,WOS:000291123500010,1,0,0,1,0,0,ai,2017,2011,2
113,WOS:000416790500071-WOS:000413946600014,WOS:000416790500071,WOS:000413946600014,0,0,1,1,0,0,iot,2017,2017,4
150,WOS:000372644300043-WOS:000290536400012,WOS:000372644300043,WOS:000290536400012,1,0,0,1,0,0,ai,2016,2011,1
161,WOS:000408751100012-WOS:000400822900194,WOS:000408751100012,WOS:000400822900194,1,0,0,1,0,0,ai,2017,2017,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003829,WOS:000398223000001-WOS:000385464600002,WOS:000398223000001,WOS:000385464600002,0,0,1,0,0,1,iot,2017,2016,82
1004061,WOS:000407062300038-WOS:000388487400055,WOS:000407062300038,WOS:000388487400055,0,0,1,0,0,1,iot,2016,2016,0
1004062,WOS:000392263300009-WOS:000388487400055,WOS:000392263300009,WOS:000388487400055,0,0,1,0,0,1,iot,2016,2016,1
1004077,WOS:000403205200025-WOS:000383222000014,WOS:000403205200025,WOS:000383222000014,0,0,1,0,0,1,iot,2017,2016,2


In [78]:
for index, row in df5.iterrows():
    AIsource = row['s.ai']
    AItarget = row['t.ai']
    ROBsource = row['s.rob']
    ROBtarget = row['t.rob']
    IOTsource = row['s.iot']
    IOTtarget = row['t.iot']
    srcYear = row['Year']#wosYears[row[1]]
    trgYear = row['TargetYear']
    if ((AIsource == 1) & (ROBtarget == 1)):
        #AItoROB[(srcYear, trgYear)] = 1
        keys = (srcYear, trgYear)
        AItoROB[keys] = AItoROB.setdefault(keys, 0) + 1
       # print(keys)
        #if AItoROB.keys in ((srcYear, trgYear)):
           # print(srcYear)
         #   AItoROB[(srcYear, trgYear)] += 1
            
       # else: AItoROB[(srcYear, trgYear)] = 1

    if ((ROBsource == 1) & (IOTtarget == 1)):
        #print((srcYear, trgYear))
        keys = (srcYear, trgYear)
        ROBtoIOT[keys] = ROBtoIOT.setdefault(keys, 0) + 1
       # if ROBtoIOT.keys in ((srcYear, trgYear)):
        #    ROBtoIOT[(srcYear, trgYear)] += 1
        #else: ROBtoIOT[(srcYear, trgYear)] = 1
    if ((IOTsource == 1) & (AItarget == 1)):
        #if IOTtoAI.keys in ((srcYear, trgYear)):
        ##    IOTtoAI[(srcYear, trgYear)] += 1
       # else: IOTtoAI[(srcYear, trgYear)] = 1
        keys = (srcYear, trgYear)
        IOTtoAI[keys] = IOTtoAI.setdefault(keys, 0) + 1
    if ((ROBsource == 1) & (AItarget == 1)):
       # if ROBtoAI.keys in ((srcYear, trgYear)):
       #     ROBtoAI[(srcYear, trgYear)] += 1
       # else: ROBtoAI[(srcYear, trgYear)] = 1
        keys = (srcYear, trgYear)
        ROBtoAI[keys] = ROBtoAI.setdefault(keys, 0) + 1
    if ((AIsource == 1) & (IOTtarget == 1)):
       # if AItoIOT.keys in ((srcYear, trgYear)):
        #    AItoIOT[(srcYear, trgYear)] += 1
       # else: AItoIOT[(srcYear, trgYear)] = 1
        keys = (srcYear, trgYear)
        AItoIOT[keys] = AItoIOT.setdefault(keys, 0) + 1
    if ((IOTsource == 1) & (ROBtarget == 1)):
      #  if IOTtoROB.keys in ((srcYear, trgYear)):
       #     IOTtoROB[(srcYear, trgYear)] += 1
      #  else: IOTtoROB[(srcYear, trgYear)] = 1
        keys = (srcYear, trgYear)
        IOTtoROB[keys] = IOTtoROB.setdefault(keys, 0) + 1
   # print(row['c1'], row['c2'])


In [81]:
ROBtoIOT

{(2016, 2014): 4,
 (2016, 2010): 1,
 (2017, 2010): 1,
 (2015, 2010): 1,
 (2016, 2015): 4,
 (2017, 2016): 3,
 (2016, 2012): 3,
 (2011, 2010): 2,
 (2017, 2015): 3,
 (2016, 2013): 2,
 (2014, 2013): 1,
 (2016, 2016): 1,
 (2017, 2013): 2}

In [82]:
IOTtoROB

{(2014, 2011): 2,
 (2017, 2009): 3,
 (2017, 2013): 8,
 (2016, 2013): 2,
 (2013, 2013): 1,
 (2016, 2015): 4,
 (2017, 2015): 9,
 (2017, 2004): 2,
 (2016, 2010): 2,
 (2017, 2016): 5,
 (2014, 2007): 2,
 (2013, 2007): 1,
 (2016, 2006): 1,
 (2017, 2012): 1,
 (2016, 2011): 2,
 (2015, 2000): 1,
 (2017, 2014): 6,
 (2016, 2008): 4,
 (2017, 2003): 1,
 (2016, 2012): 2,
 (2017, 2017): 2,
 (2017, 2005): 2,
 (2015, 2001): 1,
 (2017, 2011): 3,
 (2014, 2013): 1,
 (2016, 2007): 1,
 (2015, 2004): 2,
 (2016, 2002): 1,
 (2014, 2010): 2,
 (2016, 2009): 1,
 (2016, 2014): 2,
 (2016, 2000): 1,
 (2017, 2008): 1,
 (2012, 2012): 1,
 (2017, 2010): 1,
 (2015, 2015): 1,
 (2016, 2016): 1,
 (2015, 2013): 1,
 (2015, 2005): 1,
 (2015, 2011): 1}

In [83]:
AItoIOT

{(2017, 2010): 1,
 (2015, 2010): 1,
 (2016, 2013): 3,
 (2017, 2014): 2,
 (2017, 2016): 1,
 (2015, 2014): 8,
 (2016, 2012): 1,
 (2014, 2013): 2,
 (2014, 2011): 1,
 (2011, 2013): 1,
 (2017, 2013): 2,
 (2015, 2008): 1,
 (2016, 2014): 3,
 (2016, 2015): 1,
 (2016, 2016): 2,
 (2014, 2012): 3,
 (2014, 2014): 1,
 (2014, 2015): 1,
 (2017, 2012): 1,
 (2015, 2011): 1,
 (2017, 2015): 1}

In [84]:
AItoROB

{(2010, 2004): 1,
 (2016, 2014): 3,
 (2016, 2013): 6,
 (2017, 2013): 9,
 (2014, 2002): 1,
 (2014, 2011): 7,
 (2003, 2001): 2,
 (2013, 2008): 1,
 (2014, 2007): 2,
 (2007, 2005): 3,
 (2017, 2005): 6,
 (2013, 2011): 6,
 (2017, 2009): 5,
 (2013, 2006): 3,
 (2011, 2007): 4,
 (2017, 2015): 6,
 (2009, 2005): 2,
 (2008, 2005): 4,
 (2012, 2009): 7,
 (2010, 2007): 4,
 (2016, 2007): 3,
 (2015, 2006): 2,
 (2013, 2002): 1,
 (2011, 2002): 1,
 (2008, 1998): 1,
 (2014, 2009): 3,
 (2009, 2009): 1,
 (2012, 1999): 2,
 (2002, 1999): 1,
 (2017, 2007): 7,
 (2013, 2007): 2,
 (2009, 2007): 4,
 (2014, 2013): 2,
 (2015, 2012): 3,
 (2017, 2000): 1,
 (2008, 2000): 1,
 (2012, 2011): 2,
 (2013, 2003): 1,
 (2013, 2001): 1,
 (2008, 2001): 1,
 (2017, 2004): 1,
 (2015, 2004): 3,
 (2011, 2008): 8,
 (2007, 1998): 2,
 (2017, 2011): 2,
 (2003, 1999): 1,
 (2016, 2006): 3,
 (2014, 2004): 2,
 (2011, 2000): 2,
 (2004, 2000): 1,
 (2003, 2000): 1,
 (2005, 1998): 1,
 (2017, 2010): 2,
 (2017, 2014): 1,
 (2014, 2014): 1,
 (2016, 20

In [85]:
ROBtoAI

{(2011, 2010): 1,
 (2017, 2017): 2,
 (2017, 2016): 4,
 (2014, 2009): 1,
 (2016, 2015): 2,
 (2017, 2015): 2,
 (2016, 2014): 2,
 (2014, 2013): 1,
 (2017, 2013): 4,
 (2015, 2013): 4,
 (2012, 2009): 4,
 (2017, 2009): 4,
 (2016, 2009): 2,
 (2016, 2013): 2,
 (2017, 2012): 6,
 (2013, 2012): 1,
 (2015, 2012): 3,
 (2016, 2012): 5,
 (2016, 2011): 2,
 (2014, 2011): 1,
 (2017, 2011): 1,
 (2015, 2010): 2,
 (2016, 2010): 3,
 (2014, 2010): 1,
 (2013, 2010): 1,
 (2012, 2010): 1,
 (2010, 2009): 3,
 (2015, 2009): 2,
 (2013, 2009): 1,
 (2011, 2009): 1,
 (2010, 2008): 3,
 (2009, 2008): 2,
 (2012, 2008): 2,
 (2011, 2008): 2,
 (2016, 2008): 2,
 (2015, 2008): 1,
 (2013, 2008): 1,
 (2017, 2008): 1,
 (2013, 2007): 1,
 (2015, 2007): 1,
 (2011, 2007): 1,
 (2010, 2007): 3,
 (2016, 2006): 2,
 (2012, 2006): 1,
 (2010, 2006): 1,
 (2011, 2006): 1,
 (2014, 2006): 1,
 (2017, 2006): 4,
 (2009, 2006): 1,
 (2015, 2005): 2,
 (2017, 2005): 1,
 (2009, 2005): 1,
 (2007, 2005): 1,
 (2013, 2005): 2,
 (2012, 2005): 1,
 (2017, 20

In [86]:
IOTtoAI

{(2017, 2017): 2,
 (2015, 2011): 2,
 (2016, 2016): 3,
 (2017, 2016): 5,
 (2016, 2015): 3,
 (2017, 2015): 4,
 (2017, 2009): 6,
 (2016, 2014): 3,
 (2015, 2014): 4,
 (2017, 2014): 3,
 (2015, 2009): 5,
 (2017, 2010): 2,
 (2017, 2013): 2,
 (2015, 2013): 1,
 (2016, 2013): 2,
 (2017, 2012): 4,
 (2016, 2012): 1,
 (2015, 2012): 3,
 (2016, 2011): 3,
 (2013, 2011): 1,
 (2017, 2011): 2,
 (2014, 2010): 1,
 (2016, 2010): 2,
 (2012, 2009): 4,
 (2013, 2009): 1,
 (2016, 2009): 2,
 (2014, 2009): 2,
 (2015, 2008): 1,
 (2015, 2007): 2,
 (2017, 2007): 1,
 (2011, 2007): 1,
 (2016, 2006): 1,
 (2017, 2005): 2,
 (2017, 2003): 1,
 (2016, 2003): 1,
 (2013, 2003): 1,
 (2012, 2001): 1,
 (2017, 2000): 1,
 (2013, 2000): 1,
 (2016, 1999): 1,
 (2017, 1999): 1}

In [79]:
for key, value in ROBtoIOT.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in IOTtoROB.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 20, srcLon]) #IoT row
    Nodes.append([len(Nodes) + 1, int(key[1]), 40, trgLon]) #ROB row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in AItoIOT.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 60, srcLon]) #AI row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in IOTtoAI.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 20, srcLon]) #IoT row
    Nodes.append([len(Nodes) + 1, int(key[1]), 60, trgLon]) #AI row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in AItoROB.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 60, srcLon]) #AI row
    Nodes.append([len(Nodes) + 1, int(key[1]), 40, trgLon]) #ROB row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in ROBtoAI.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 60, trgLon]) #AI row
    Edges.append([len(Nodes) - 1, len(Nodes), value])

for node in Nodes:
    NodeDict[node[0]] = node[2] # create dictionary of ID-Latitude pairs

with open('WOS_convergence_edges.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["source", "target", "weight", "cat"])
    for edge in Edges:
        if (NodeDict[edge[0]] == 20): # IoT
            edge.append(0)
            writer.writerow(edge)
        elif (NodeDict[edge[0]] == 40): # Robotics
            edge.append(1)
            writer.writerow(edge)
        else:                          # AI
            edge.append(2)
            writer.writerow(edge)

with open('WOS_convergence_nodes.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "label", "lat", "lon"])
    for node in Nodes:
        writer.writerow(node)

In [87]:
Edges

[[1, 2, 4, 1],
 [3, 4, 1, 1],
 [5, 6, 1, 1],
 [7, 8, 1, 1],
 [9, 10, 4, 1],
 [11, 12, 3, 1],
 [13, 14, 3, 1],
 [15, 16, 2, 1],
 [17, 18, 3, 1],
 [19, 20, 2, 1],
 [21, 22, 1, 1],
 [23, 24, 1, 1],
 [25, 26, 2, 1],
 [27, 28, 2, 0],
 [29, 30, 3, 0],
 [31, 32, 8, 0],
 [33, 34, 2, 0],
 [35, 36, 1, 0],
 [37, 38, 4, 0],
 [39, 40, 9, 0],
 [41, 42, 2, 0],
 [43, 44, 2, 0],
 [45, 46, 5, 0],
 [47, 48, 2, 0],
 [49, 50, 1, 0],
 [51, 52, 1, 0],
 [53, 54, 1, 0],
 [55, 56, 2, 0],
 [57, 58, 1, 0],
 [59, 60, 6, 0],
 [61, 62, 4, 0],
 [63, 64, 1, 0],
 [65, 66, 2, 0],
 [67, 68, 2, 0],
 [69, 70, 2, 0],
 [71, 72, 1, 0],
 [73, 74, 3, 0],
 [75, 76, 1, 0],
 [77, 78, 1, 0],
 [79, 80, 2, 0],
 [81, 82, 1, 0],
 [83, 84, 2, 0],
 [85, 86, 1, 0],
 [87, 88, 2, 0],
 [89, 90, 1, 0],
 [91, 92, 1, 0],
 [93, 94, 1, 0],
 [95, 96, 1, 0],
 [97, 98, 1, 0],
 [99, 100, 1, 0],
 [101, 102, 1, 0],
 [103, 104, 1, 0],
 [105, 106, 1, 0],
 [107, 108, 1, 2],
 [109, 110, 1, 2],
 [111, 112, 3, 2],
 [113, 114, 2, 2],
 [115, 116, 1, 2],
 [117,

In [37]:
print("AI->ROB: " + str(AItoROB))
#print("ROB->IOT: " + str(ROBtoIOT))
#print("IOT->AI: " + str(IOTtoAI))
#print("ROB->AI: " + str(ROBtoAI))
#print("AI->IOT: " + str(AItoIOT))
#print("IOT->ROB : " + str(IOTtoROB))

AI->ROB: {}


In [46]:
srcYear = dict(zip(df5.id, df5.Year))
trgYear = dict(zip(df5.id, df5.TargetYear))

In [47]:
if ((AIsource == '1') & (ROBtarget == '1')):
    if AItoROB.has_key((srcYear, trgYear)):
        AItoROB[(srcYear, trgYear)] += 1
    else: AItoROB[(srcYear, trgYear)] = 1
if ((ROBsource == '1') & (IOTtarget == '1')):
    if ROBtoIOT.has_key((srcYear, trgYear)):
        ROBtoIOT[(srcYear, trgYear)] += 1
    else: ROBtoIOT[(srcYear, trgYear)] = 1
if ((IOTsource == '1') & (AItarget == '1')):
    if IOTtoAI.has_key((srcYear, trgYear)):
        IOTtoAI[(srcYear, trgYear)] += 1
    else: IOTtoAI[(srcYear, trgYear)] = 1
if ((ROBsource == '1') & (AItarget == '1')):
    if ROBtoAI.has_key((srcYear, trgYear)):
        ROBtoAI[(srcYear, trgYear)] += 1
    else: ROBtoAI[(srcYear, trgYear)] = 1
if ((AIsource == '1') & (IOTtarget == '1')):
    if AItoIOT.has_key((srcYear, trgYear)):
        AItoIOT[(srcYear, trgYear)] += 1
    else: AItoIOT[(srcYear, trgYear)] = 1
if ((IOTsource == '1') & (ROBtarget == '1')):
    if IOTtoROB.has_key((srcYear, trgYear)):
        IOTtoROB[(srcYear, trgYear)] += 1
    else: IOTtoROB[(srcYear, trgYear)] = 1

In [50]:
for row in df5:
    if ((AIsource == '1') & (ROBtarget == '1')):
        if AItoROB.has_key((srcYear, trgYear)):
            AItoROB[(srcYear, trgYear)] += 1
        else: AItoROB[(srcYear, trgYear)] = 1

In [51]:
for key, value in ROBtoIOT.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in IOTtoROB.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 20, srcLon]) #IoT row
    Nodes.append([len(Nodes) + 1, int(key[1]), 40, trgLon]) #ROB row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in AItoIOT.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 60, srcLon]) #AI row
    Nodes.append([len(Nodes) + 1, int(key[1]), 20, trgLon]) #IoT row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in IOTtoAI.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 20, srcLon]) #IoT row
    Nodes.append([len(Nodes) + 1, int(key[1]), 60, trgLon]) #AI row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in AItoROB.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 60, srcLon]) #AI row
    Nodes.append([len(Nodes) + 1, int(key[1]), 40, trgLon]) #ROB row
    Edges.append([len(Nodes) - 1, len(Nodes), value])
for key, value in ROBtoAI.items():
    srcLon = (int(key[0]) - 1998) * 10
    trgLon = (int(key[1]) - 1998) * 10
    Nodes.append([len(Nodes) + 1, int(key[0]), 40, srcLon]) #Robot row
    Nodes.append([len(Nodes) + 1, int(key[1]), 60, trgLon]) #AI row
    Edges.append([len(Nodes) - 1, len(Nodes), value])

for node in Nodes:
    NodeDict[node[0]] = node[2] # create dictionary of ID-Latitude pairs

with open('WOS_convergence_edges.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["source", "target", "weight", "cat"])
    for edge in Edges:
        if (NodeDict[edge[0]] == 20): # IoT
            edge.append(0)
            writer.writerow(edge)
        elif (NodeDict[edge[0]] == 40): # Robotics
            edge.append(1)
            writer.writerow(edge)
        else:                          # AI
            edge.append(2)
            writer.writerow(edge)

with open('WOS_convergence_nodes.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "label", "lat", "lon"])
    for node in Nodes:
        writer.writerow(node)